In [60]:
import pandas as pd
import numpy as np

# GUI - Under Construction

This creates the dialogue box where the user enters the project information

Needs to be able to import the customer load file from csv
Inputs:
Cost of solar (dollar per w), Cost of storage (dollar per kWh)

In [61]:
import tkinter
from tkinter import *
fields = 'Storage Size kW','Lat/Lon','PV Cost per Watt','Diesel Fuel Cost','Diesel Size kW','Interest Rate','Tenor','Electricity Price'

def fetch(entries):
    global storedfields
    storedfields = {}
    for entry in entries:
        field = entry[0]
        text=entry[1].get()
        storedfields[field] = text
        print('%s:"%s"'%(field,text))
        
def makeform(root,fields):
    entries=[]
    for field in fields:
        row=Frame(root)
        lab=Label(row,width=30,text=field,anchor='w')
        ent=Entry(row)
        row.pack(side=TOP,fill=X,padx=5,pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT,expand=YES,fill=X)
        entries.append((field,ent))
    return entries
    
if __name__ == '__main__':
    root=Tk()
    ents=makeform(root,fields)
    root.bind('<Return>'),(lambda event,e=ents:fetch(e))
    b1=Button(root,text='Submit',command=(lambda e=ents:fetch(e)))
    b1.pack(side=LEFT,padx=5,pady=5)
    b2=Button(root,text='Complete',command=root.destroy)
    b2.pack(side=LEFT,padx=5,pady=5)
    root.mainloop()

inputvariables = pd.DataFrame.from_dict(storedfields,orient='index') 
inputvariables.columns = ['values']
inputvariables = inputvariables.replace('',0) 
inputvariables.values = inputvariables.values.astype(float) 

Storage Size kW:"250"
Lat/Lon:"3040"
PV Cost per Watt:"3.5"
Diesel Fuel Cost:"3.5"
Diesel Size kW:"250"
Interest Rate:".10"
Tenor:"10"
Electricity Price:".40"


# Import Datasets: PV Output and Customer Load

Note: Need to find data that better suits this scenario
Note2: Need to automate the PV data pull here
Note3: Make customer load read from excel

In [74]:
#PV Output
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer Load
CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

# Scenario Inputs: Diesel, Storage, PV, and Electricity Rates

Note: Need to get all this into Tkinter
Note2: Need to add in dynamic approach for lifetime on diesel and storage based on throughput
Note3: Need to add in roundtrip efficiency
Note4: Add in solar costs and multi-year

In [75]:
#Input from Model

storage_size_kW = float(inputvariables.iloc[0])
lat_lon = float(inputvariables.iloc[1])
pv_cpw = float(inputvariables.iloc[2])
diesel_fuel_cost = float(inputvariables.iloc[3]) 
diesel_capex = float(inputvariables.iloc[4])
interest_rate = float(inputvariables.iloc[5])
tenor = float(inputvariables.iloc[6])
electricity_price = float(inputvariables.iloc[7])


In [76]:
# Fixed variables

#Storage
storage_hours = float(4)
storage_capex = float(250000)
years_of_use = float(10)
storage_max_throughput = float(2700*storage_size_kW*storage_hours)

#Storage Annual Payment
storage_annual_payment = (storage_capex *(interest_rate * (1 + interest_rate) ** tenor) /
                         ((1 + interest_rate) ** tenor - 1))

#solar LCOE
pv_size_kW = 150
pv_capex = (pv_size_kW * pv_cpw * 1000)
pv_irradiation = 1400
pv_years = 20
pv_total_production = pv_irradiation * pv_years * pv_size_kW

#Solar LCOE
solar_lcoe = pv_capex / pv_total_production

#Diesel Generator
diesel_price = float(.40)
diesel_years = 10
diesel_max_throughput = float(36000) #hours
    
#Electricity Price - TOU
        #00:01 - 11:00
        #11:01 - 15:00
        #15:01 - 24:00

# Clean Datasets and Create New Dataframe

In [77]:
#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']
EnergyMaster = CustomerLoad

# Create Required Columns for New Dataframe

In [78]:
#Add electricity price
EnergyMaster['electricity_price'] = electricity_price

#create column for pv production
EnergyMaster['PV_Output'] = PVOutput

#create column to show customers load after they have consumed PV
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#create column for amount of PV left after customer load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#create storage state of charge in three parts

    #charge storage first from the remaining PV available
EnergyMaster['starting_storage_SOC_kWh'] = EnergyMaster['PV_Output_Post_Load'].cumsum().clip(0,storage_size_kW * storage_hours)
    
    #identify the potential charge from the grid capped by the storage units size
EnergyMaster['potential_grid_charge'] = ((storage_size_kW * storage_hours) - EnergyMaster['starting_storage_SOC_kWh']).cumsum().clip(0,storage_size_kW)
    
    #charge storage from the grid capped at the storage unit size
EnergyMaster['finished_storage_SOC_kWh'] = (EnergyMaster['starting_storage_SOC_kWh'] + EnergyMaster['potential_grid_charge']).cumsum().clip(0,storage_size_kW * storage_hours)

    #create customers load post storage
EnergyMaster['customer_load_post_storage'] = (EnergyMaster['Customer_Load_Post_PV'] - EnergyMaster['starting_storage_SOC_kWh']).clip_lower(0)

    #show discharged storage amount
EnergyMaster['storage_discharge'] = EnergyMaster['Customer_Load_Post_PV'] - EnergyMaster['customer_load_post_storage']

    #show charge storage amount
EnergyMaster['storage_charge'] = EnergyMaster['finished_storage_SOC_kWh'] - EnergyMaster['starting_storage_SOC_kWh']

    #show new customer load with storage charging
#EnergyMaster['final_customer_load'] = 
    

#Add solar LCOE
EnergyMaster['solar_lcoe'] = solar_lcoe

#Add diesel state of charge
#EnergyMaster['Diesel_SOC'] = EnergyMaster['CustomerLoad'].cumsum().clip(0,diesel_size_kW)

EnergyMaster.head(100)

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,starting_storage_SOC_kWh,potential_grid_charge,finished_storage_SOC_kWh,customer_load_post_storage,storage_discharge,storage_charge,solar_lcoe
0,22.8125,2014-01-01 00:00:00,0.4,0.000,0.0000,22.8125,0.0000,250.0,250.0,22.8125,0.0000,250.0000,0.125
1,24.6650,2014-01-01 00:00:00,0.4,0.000,0.0000,24.6650,0.0000,250.0,500.0,24.6650,0.0000,500.0000,0.125
2,24.1725,2014-01-01 01:00:00,0.4,0.000,0.0000,24.1725,0.0000,250.0,750.0,24.1725,0.0000,750.0000,0.125
3,24.9500,2014-01-01 01:00:00,0.4,0.000,0.0000,24.9500,0.0000,250.0,1000.0,24.9500,0.0000,1000.0000,0.125
4,23.6175,2014-01-01 01:00:00,0.4,0.000,0.0000,23.6175,0.0000,250.0,1000.0,23.6175,0.0000,1000.0000,0.125
5,23.7425,2014-01-01 01:00:00,0.4,0.000,0.0000,23.7425,0.0000,250.0,1000.0,23.7425,0.0000,1000.0000,0.125
6,24.7725,2014-01-01 02:00:00,0.4,0.000,0.0000,24.7725,0.0000,250.0,1000.0,24.7725,0.0000,1000.0000,0.125
7,23.4475,2014-01-01 02:00:00,0.4,19.396,0.0000,4.0515,0.0000,250.0,1000.0,4.0515,0.0000,1000.0000,0.125
8,24.4975,2014-01-01 02:00:00,0.4,230.275,205.7775,0.0000,205.7775,250.0,1000.0,0.0000,0.0000,794.2225,0.125
9,24.4175,2014-01-01 02:00:00,0.4,433.377,408.9595,0.0000,614.7370,250.0,1000.0,0.0000,0.0000,385.2630,0.125


# Create Randomized Grid Outages

In [87]:
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

EnergyMaster.head()

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,starting_storage_SOC_kWh,potential_grid_charge,finished_storage_SOC_kWh,customer_load_post_storage,storage_discharge,storage_charge,solar_lcoe,grid_down_probability,grid_down,cost_in_diesel,cost_in_solar
0,22.8125,2014-01-01 00:00:00,0.4,0.0,0.0,22.8125,0.0,250.0,250.0,22.8125,0.0,250.0,0.125,0.681953,0,0.4,0.125
1,24.6650,2014-01-01 00:00:00,0.4,0.0,0.0,24.6650,0.0,250.0,500.0,24.6650,0.0,500.0,0.125,0.960971,0,0.4,0.125
2,24.1725,2014-01-01 01:00:00,0.4,0.0,0.0,24.1725,0.0,250.0,750.0,24.1725,0.0,750.0,0.125,0.793674,0,0.4,0.125
3,24.9500,2014-01-01 01:00:00,0.4,0.0,0.0,24.9500,0.0,250.0,1000.0,24.9500,0.0,1000.0,0.125,0.334094,0,0.4,0.125
4,23.6175,2014-01-01 01:00:00,0.4,0.0,0.0,23.6175,0.0,250.0,1000.0,23.6175,0.0,1000.0,0.125,0.636540,0,0.4,0.125


# Calculate Combined Price of Diesel Scenario

Note: Storage-only scenario cost is the annual payment lease payment plus electricity price
Note2: Storage and PV cost is the annual lease payment plus the cost of solar plus electricity price

In [91]:
#diesel scenario

EnergyMaster['cost_in_diesel'] = diesel_price
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']].sum()
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']]
EnergyMaster['diesel_scenario_price'].fillna(EnergyMaster['cost_in_diesel'],inplace=True)

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,starting_storage_SOC_kWh,potential_grid_charge,finished_storage_SOC_kWh,customer_load_post_storage,storage_discharge,storage_charge,solar_lcoe,grid_down_probability,grid_down,cost_in_diesel,cost_in_solar,diesel_scenario_price
0,22.8125,2014-01-01 00:00:00,0.4,0.000,0.0000,22.8125,0.0000,250.0,250.0,22.8125,0.0000,250.0000,0.125,0.681953,0,0.4,0.125,0.4
1,24.6650,2014-01-01 00:00:00,0.4,0.000,0.0000,24.6650,0.0000,250.0,500.0,24.6650,0.0000,500.0000,0.125,0.960971,0,0.4,0.125,0.4
2,24.1725,2014-01-01 01:00:00,0.4,0.000,0.0000,24.1725,0.0000,250.0,750.0,24.1725,0.0000,750.0000,0.125,0.793674,0,0.4,0.125,0.4
3,24.9500,2014-01-01 01:00:00,0.4,0.000,0.0000,24.9500,0.0000,250.0,1000.0,24.9500,0.0000,1000.0000,0.125,0.334094,0,0.4,0.125,0.4
4,23.6175,2014-01-01 01:00:00,0.4,0.000,0.0000,23.6175,0.0000,250.0,1000.0,23.6175,0.0000,1000.0000,0.125,0.636540,0,0.4,0.125,0.4
5,23.7425,2014-01-01 01:00:00,0.4,0.000,0.0000,23.7425,0.0000,250.0,1000.0,23.7425,0.0000,1000.0000,0.125,0.411814,0,0.4,0.125,0.4
6,24.7725,2014-01-01 02:00:00,0.4,0.000,0.0000,24.7725,0.0000,250.0,1000.0,24.7725,0.0000,1000.0000,0.125,0.239828,0,0.4,0.125,0.4
7,23.4475,2014-01-01 02:00:00,0.4,19.396,0.0000,4.0515,0.0000,250.0,1000.0,4.0515,0.0000,1000.0000,0.125,0.085826,0,0.4,0.125,0.4
8,24.4975,2014-01-01 02:00:00,0.4,230.275,205.7775,0.0000,205.7775,250.0,1000.0,0.0000,0.0000,794.2225,0.125,0.844453,0,0.4,0.125,0.4
9,24.4175,2014-01-01 02:00:00,0.4,433.377,408.9595,0.0000,614.7370,250.0,1000.0,0.0000,0.0000,385.2630,0.125,0.587106,0,0.4,0.125,0.4


# Calculate Combined Price of Solar Scenario

In [92]:
EnergyMaster['cost_in_solar'] = solar_lcoe
EnergyMaster['solar_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','solar_lcoe']].sum()
EnergyMaster['solar_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','solar_lcoe']]
EnergyMaster['solar_scenario_price'].fillna(EnergyMaster['solar_lcoe'],inplace=True)

# Calculate Totals

In [93]:
#sum by year
EnergyMaster['timestamp_hour'] = pd.to_datetime(EnergyMaster['timestamp_hour'])
EnergyMaster['year'] = EnergyMaster['timestamp_hour'].dt.year

TotalSolarPrice = EnergyMaster.groupby(['year'])['solar_scenario_price'].sum()
TotalDieselPrice = (EnergyMaster.groupby(['year'])['diesel_scenario_price'].sum() + (diesel_capex / diesel_years))
TotalStoragePrice = (EnergyMaster.groupby(['year'])['solar_scenario_price'].sum() + storage_annual_payment)

print()
TotalStoragePrice

year
2014    53996.923721
2015    44934.673721
Name: solar_scenario_price, dtype: float64

# Calculate Throughput 

In [ ]:
#storage change in charge

#fix variables with new storage categories above

#EnergyMaster['storage_change_charge'] = EnergyMaster['Storage_SOC'] - EnergyMaster['Storage_SOC_Post_Load']

#storage_throughput = EnergyMaster.groupby(['year'])['storage_change_charge'].sum()
#diesel_throughput = EnergyMaster.groupby(['year'])['grid_down'].sum()

print()
diesel_throughput

# Optimization

In [ ]:
from scipy import optimize

#variables we control:
pv_cpw = [1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4]
diesel_fuel_cost = range(1,4)

In [ ]:
# this creates a dataframe of all possible combinations of the variables above:

from itertools import product
costs_df = pd.DataFrame(list(product(pv_cpw,diesel_fuel_cost)), 
                     columns=['pv_cpw', 'diesel_fuel_cost'])
costs_df.head()diesel = []
storage = []

# now, for everyone of those combinations of cpw and fuel costs 
#we will calculate the energy cost for diesel and the energy cost for storage

for index, row in costs_df.iterrows():
    
    #these are the functions we ran above, but now doing them in a more structured fashion
    df = EnergyMaster.groupby(EnergyMaster.index / row['hoursdown']).sum() 
    df['grid_down_probability'] = df['kWh'].apply(lambda x: np.random.random())
    percent_downtime = row['daysdown'] / (7.0*24/row['hoursdown']) 
    df['grid_down'] = df['grid_down_probability'].apply(grid_is_down)
    kwh_grid_down = (df['grid_down']*df['kWh']).sum() #this gives us the total kWh that the grid is down

    
    #these calculate the diesel cost in this scenario for every row in the dataframe and appends to a list
    diesel_total_cost = (diesel_annual_payment + #lease costs
                     kwh_grid_down*diesel_om_kwh + #operations and maintenance
                     df['grid_down'].sum()*hoursdown*gallons_per_hour*row['costpergal']) #fuel
    diesel.append(diesel_total_cost)
        
    #now the storage costs   
    storage_total_cost = (storage_annual_payment + kwh_grid_down*storage_electricity_rate)
    storage.append(storage_total_cost)
    
costs_df['dieselcost'] = diesel
costs_df['storagecost'] = storage